In [1]:
import sys
import os.path as osp
import pygame
import numpy as np
import cv2
import sys
import time 

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
MHI_DURATION = 0.5
DEFAULT_THRESHOLD = 32
MAX_TIME_DELTA = 0.25
MIN_TIME_DELTA = 0.05
# Load Sound Effect
pygame.init()
s = s = pygame.mixer.Sound('Dong.WAV')
cap = cv2.VideoCapture(0)

In [3]:

while(1):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    m_pre = None
    n_pre = None
    
    
    
    h, w = frame.shape[:2]
    prev_frame = frame.copy()
    motion_history = np.zeros((h, w), np.float32)
    mask_map = np.zeros((h, w), np.uint8)
    timestamp = 0
    
    
    
    while(1):
        # Calculate difference btw current frame and previous frame
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        frame_diff = cv2.absdiff(frame, prev_frame)
        gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        #ret, fgmask = cv2.threshold(gray_diff, DEFAULT_THRESHOLD, 1, cv2.THRESH_BINARY)
        _,fgmask = cv2.threshold(gray_diff, DEFAULT_THRESHOLD, 255, cv2.THRESH_BINARY)
        timestamp += 1
    
        cv2.motempl.updateMotionHistory(fgmask, motion_history, timestamp, MHI_DURATION)
        mh = np.uint8(np.clip((motion_history-(timestamp-MHI_DURATION)) / MHI_DURATION, 0, 1)*255)
        kernel = np.ones((5,5))
        
        
        try:
            frame_show = frame.copy()
            
            dilate_mask = cv2.dilate(fgmask,kernel,iterations = 6)
            potential_stick = cv2.bitwise_and(frame_show,frame_show,mask = fgmask)
            
            # In order to delete hand effect, we firstlt detect hand with skin detector
            
            resColored = cv2.bitwise_and(frame_show,frame_show,mask = dilate_mask)
            
            blur = cv2.blur(resColored,(3,3), 0)
            hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
            lower_range = np.array([2,0,0])
            upper_range = np.array([16,255,255])
    
            Hand_mask = cv2.inRange(hsv,lower_range,upper_range)
            filtered = cv2.GaussianBlur(Hand_mask, (15,15), 1)
            ret,thresh = cv2.threshold(filtered, 127, 255, 0)
            DeleteHand_mask = cv2.dilate(thresh,kernel,iterations = 10)
            
            [x_coor,y_coor] = np.where(DeleteHand_mask==255)
            
            dilate_mask[x_coor,y_coor] = 0
            
            
            # Draw the contours with maximum area
            _,contours,hierarchy= cv2.findContours(dilate_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            areas = [cv2.contourArea(c) for c in contours]
            if np.max(areas) > 1200:
                max_index = np.argmax(areas)
                cnt=contours[max_index]
                for c in cnt:
                    cv2.drawContours(frame_show, [c], 0, (0,255,0), 3)
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.rectangle(frame_show,(x,y),(x+w,y+h),(0,255,0),2)
                m = int(x+ 0.5*w)
                n = int(y+ 0.5*h)
                cv2.circle(frame_show,(m,n), 25, (255,0,0), 10)
            
                
                # Set Flag and Play Sound Effect
                if m_pre is None and n_pre is None:
                    m_pre = m
                    n_pre = n
                    
                else:
                    #if (n_pre-n>30):
                    #    cv2.putText(frame_show,"Go Up", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                    if (n_pre-n<-40):
                        cv2.putText(frame_show,"Hit", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                        s.play()
                        
                    #print(n)
                    n_pre = n
                
                
                

            cv2.imshow('raw',frame_show)
            #cv2.imshow('mhi',label_hue)
            cv2.imshow('im',resColored)
            cv2.imshow('DeleteHand_mask',DeleteHand_mask)
            
        except:
            pass
        
        
        
        #cv2.imshow('motion', mg_mask)
        prev_frame = frame.copy()
        cv2.imshow('dilate_mask', dilate_mask)
        
        
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    break
        
cv2.destroyAllWindows()
cap.release()   

In [4]:
cv2.destroyAllWindows()
cap.release()   

In [5]:
flag = [0,0]

In [180]:
flag[0] = flag[0] ^ 1

In [181]:
flag

[1, 0]